## Canu Assembly

Bin's friend is working with MinION and recommended that Canu (a fork of Celera) is the best assembler that he's used. 

https://github.com/marbl/canu

In [ ]:
cd /data1/share/scratch/minion-canu

/opt/bioinformatics-software/minion-software/canu/Linux-amd64/bin/canu \
-p flu-11-9-canu \
-d flu-11-9-canu-out \
genomeSize=32k \
-nanopore-raw flu-11-09.fastq

There weren't any contigs produced, but this did perform some error correction with falcon sense (which is a super pain to install), so I can use those reads to see if it helps with anyhting.

First I'm going to separate the FluB reads out. I assembled the entire 11-9 run which is probably a bad idea. Using the FluDB blast results I will grab the reads that map to FluB and start by assembling the full genome and if that doesn't work then I will separate them out by segment.

### Single genome assembly

WE'll start with the 11-9 FluB genome and see how that goes. Note that the PB1 segment will look wonky due to the lab strain spike in.

In [ ]:
cd /data1/share/scratch/minion-canu/flub-only

grep -B6 'Influenza B' flu-11-9.2d.fludb.xml | grep 'Iteration_query-def' | perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-11-9.2d.fastq  > flu-11-9.2d.flub-reads.fastq

/opt/bioinformatics-software/minion-software/canu/Linux-amd64/bin/canu -p flu-11-9-flub-only-canu -d flu-11-9-flub-only-canu-out genomeSize=12k -nanopore-raw flu-11-9.2d.flub-reads.fastq

This resulted in 62 corrected reads and 61 unassembled reads.

It seems like an actual assembly won't be very likely since it will try to stitch together a fragmented genome, though the error correction part may be what we should hone in on. 

Also, they do suggest that we run nanopolish prior to assembling even though they implement falcon-sense so I can give that a shot.

They also provide some documentation on how to assemble low coverage genomes that I will try since this is low coverage, especially after the error correction step (dealing with 61 reads!).
http://canu.readthedocs.org/en/stable/quick-start.html#assembling-low-coverage-datasets

I'll start with the defaults for now.

In [ ]:
cd /data1/share/scratch/minion-canu/flub-only

/opt/bioinformatics-software/minion-software/canu/Linux-amd64/bin/canu -p flu-11-9-flub-only-low-cov-canu \
-d flu-11-9-flub-only-low-cov-canu-out genomeSize=12k corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 \
minOverlapLength=499 corMaxEvidenceErate=0.3 -nanopore-raw flu-11-9.2d.flub-reads.fastq

This seemed to not work as well. There are more error corrected reads which isn't what I expected. Before jumping to combining all FluB runs I want to see if the results differ when trying to assemble one specific segment.

In [ ]:
cd /data1/share/scratch/minion-canu/flub-only

grep -P -B6 'Influenza B.+Segment:4' flu-11-9.2d.fludb.xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-11-9.2d.fastq  > flu-11-9.2d.flub-HA-reads.fastq

/opt/bioinformatics-software/minion-software/canu/Linux-amd64/bin/canu -p flu-11-9-flub-HA-only-canu \
-d flu-11-9-flub-HA-only-canu-out genomeSize=12k -nanopore-raw flu-11-9.2d.flub-HA-reads.fastq

This didn't work either. I find it very odd that when trying to assemble the whole FluB genome and the HA segment only that there are the same number of unitigs. This definitely seems like there's some weird partitioning going on here so I need to find out what the parameters are that could be causing this.

Also, I'm going to try the error correction with nanopolish first, both the consensus and eventalign modules. This will allow me to see if either of them produce assembled enough data as well as whether those feed into canu nicely. 

### Nanopolish with Canu

Nanopolish requires an alignment of some sort so not sure if this is a good idea since the reference base bias that I uncovered may have an adverse effect on the data. I will try it with the top blast segments first - this will allow me to see if this can resolve the reference base bias.

In [ ]:
cd /data1/share/scratch/minion-canu/flub-only/nanopolish

ln -s ~/projects/MinION-notebook/clinical-analysis/consensus-comparison/flub/flu-11-9.2d.11-9-top-seg-hits.sort.bam
ln -s ~/projects/MinION-notebook/clinical-analysis/consensus-comparison/flub/flu-11-9.2d.11-9-top-seg-hits.sort.bam.bai
ln -s ~/projects/MinION-notebook/clinical-analysis/consensus-comparison/flub/11-9-top-seg-hits.fasta
ln -s ../flu-11-9.2d.fastq

samtools view -F 4 flu-11-9.2d.11-9-top-seg-hits.sort.bam | cut -f1 | grep --no-group-separator \
-A3 -F -f - flu-11-9.2d.fastq > flu-11-9.2d.11-9-top-seg-hits.flub-only.fastq

perl ~/custom-scripts/fq2fa.pl flu-11-9.2d.11-9-top-seg-hits.flub-only.fastq > flu-11-9.2d.11-9-top-seg-hits.flub-only.fasta

#had to change the makefile to be compatible with the new version of samtools

make -f /opt/bioinformatics-software/minion-software/nanopolish/scripts/consensus.make \
READS=flu-11-9.2d.11-9-top-seg-hits.flub-only.fasta ASSEMBLY=11-9-top-seg-hits.fasta



### Canu Falcon Sense Corrected Reads